In [4]:
import numpy as np
import matplotlib.pyplot as plt

from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Druckrohrleitung.Druckrohrleitung_class_file import Druckrohrleitung_class
from Turbinen.Turbinen_class_file import Francis_Turbine
from Regler.Regler_class_file import PI_controller_class

In [5]:
# define constants

    # for physics
g                   = 9.81                                          # [m/s²]    gravitational acceleration 
rho                 = 1000.                                         # [kg/m³]   density of water 
pUnit_calc          = 'Pa'                                          # [text]    DO NOT CHANGE! for pressure conversion in print statements and plot labels 
pUnit_conv          = 'mWS'                                         # [text]    for pressure conversion in print statements and plot labels

    # for Turbine
Tur_Q_nenn          = 0.85                                           # [m³/s]    nominal flux of turbine 
Tur_p_nenn          = pressure_conversion(10.6,'bar',pUnit_calc)     # [Pa]      nominal pressure of turbine 
Tur_closingTime     = 90.                                            # [s]       closing time of turbine

    # for PI controller
Con_targetLevel     = 8.                                            # [m]
Con_K_p             = 0.1                                           # [-]       proportional constant of PI controller
Con_T_i             = 1000.                                         # [s]       timespan in which a steady state error is corrected by the intergal term
Con_deadbandRange   = 0.05                                          # [m]       Deadband range around targetLevel for which the controller does NOT intervene

    # for pipeline
Pip_length          = (535.+478.)                                   # [m]       length of pipeline
Pip_dia             = 0.9                                           # [m]       diameter of pipeline
Pip_area            = Pip_dia**2/4*np.pi                            # [m²]      crossectional area of pipeline
Pip_head            = 105.                                          # [m]       hydraulic head of pipeline without reservoir
Pip_angle           = np.arcsin(Pip_head/Pip_length)                # [rad]     elevation angle of pipeline 
Pip_n_seg           = 50                                            # [-]       number of pipe segments in discretization
Pip_f_D             = 0.014                                         # [-]       Darcy friction factor
Pip_pw_vel          = 500.                                          # [m/s]     propagation velocity of the pressure wave (pw) in the given pipeline
    # derivatives of the pipeline constants
Pip_dx              = Pip_length/Pip_n_seg                          # [m]       length of each pipe segment
Pip_dt              = Pip_dx/Pip_pw_vel                             # [s]       timestep according to method of characteristics
Pip_nn              = Pip_n_seg+1                                   # [1]       number of nodes
Pip_x_vec           = np.arange(0,Pip_nn,1)*Pip_dx                  # [m]       vector holding the distance of each node from the upstream reservoir along the pipeline
Pip_h_vec           = np.arange(0,Pip_nn,1)*Pip_head/Pip_n_seg      # [m]       vector holding the vertival distance of each node from the upstream reservoir

    # for reservoir
Res_area_base       = 74.                                           # [m²]      total base are of the cuboid reservoir   
Res_area_out        = Pip_area                                      # [m²]      outflux area of the reservoir, given by pipeline area
Res_level_crit_lo   = 0.                                            # [m]       for yet-to-be-implemented warnings
Res_level_crit_hi   = np.inf                                        # [m]       for yet-to-be-implemented warnings
Res_dt_approx       = 1e-3                                          # [s]       approx. timestep of reservoir time evolution to ensure numerical stability (see Res_nt why approx.)
Res_nt              = max(1,int(Pip_dt//Res_dt_approx))             # [1]       number of timesteps of the reservoir time evolution within one timestep of the pipeline
Res_dt              = Pip_dt/Res_nt                                 # [s]       harmonised timestep of reservoir time evolution

    # for general simulation
flux_init           = Tur_Q_nenn/1.1                                # [m³/s]    initial flux through whole system for steady state initialization  
level_init          = Con_targetLevel                               # [m]       initial water level in upstream reservoir for steady state initialization
simTime_target      = 100.                                          # [s]       target for total simulation time (will vary slightly to fit with Pip_dt)
nt                  = int(simTime_target//Pip_dt)                   # [1]       Number of timesteps of the whole system
t_vec               = np.arange(0,nt+1,1)*Pip_dt                    # [s]       time vector. At each step of t_vec the system parameters are stored


In [6]:
# create objects

# Upstream reservoir
reservoir = Ausgleichsbecken_class(Res_area_base,Res_area_out,Res_dt,pUnit_conv,Res_level_crit_lo,Res_level_crit_hi,rho)
reservoir.set_steady_state(flux_init,level_init)

# pipeline
pipe = Druckrohrleitung_class(Pip_length,Pip_dia,Pip_head,Pip_n_seg,Pip_f_D,Pip_pw_vel,Pip_dt,pUnit_conv,rho)
pipe.set_steady_state(flux_init,reservoir.get_current_pressure())

# downstream turbine
turbine = Francis_Turbine(Tur_Q_nenn,Tur_p_nenn,Tur_closingTime,Pip_dt,pUnit_conv)
turbine.set_steady_state(flux_init,pipe.get_current_pressure_distribution()[-1])

# influx setting turbine
turbine_in = Francis_Turbine(Tur_Q_nenn,Tur_p_nenn,Tur_closingTime,Pip_dt,pUnit_conv)
turbine_in.set_steady_state(flux_init,Tur_p_nenn)

# level controll
level_control = PI_controller_class(Con_targetLevel,Con_deadbandRange,Con_K_p,Con_T_i,Pip_dt)
level_control.set_control_variable(turbine.get_current_LA(),display_warning=False)


In [7]:
# initialization for Timeloop

v_old = pipe.get_current_velocity_distribution()
v_min = pipe.get_current_velocity_distribution()
v_max = pipe.get_current_velocity_distribution()
Q_old = pipe.get_current_flux_distribution()
Q_min = pipe.get_current_flux_distribution()
Q_max = pipe.get_current_flux_distribution()
p_old = pipe.get_current_pressure_distribution()
p_min = pipe.get_current_pressure_distribution()
p_max = pipe.get_current_pressure_distribution()

Q_in_vec = np.zeros_like(t_vec)
Q_in_vec[0] = flux_init

v_boundary_res  = np.zeros_like(t_vec)
v_boundary_tur  = np.zeros_like(t_vec)
Q_boundary_res  = np.zeros_like(t_vec)
Q_boundary_tur  = np.zeros_like(t_vec)
p_boundary_res  = np.zeros_like(t_vec)
p_boundary_tur  = np.zeros_like(t_vec)

level_vec       = np.full_like(t_vec,level_init)  # level at the end of each pipeline timestep
volume_vec      = np.full_like(t_vec,reservoir.get_current_volume())  # volume at the end of each pipeline timestep

v_boundary_res[0]   = v_old[0]
v_boundary_tur[0]   = v_old[-1] 
Q_boundary_res[0]   = Q_old[0]
Q_boundary_tur[0]   = Q_old[-1]
p_boundary_res[0]   = p_old[0]
p_boundary_tur[0]   = p_old[-1]

LA_soll_vec = np.full_like(t_vec,turbine.get_current_LA())
LA_ist_vec  = np.full_like(t_vec,turbine.get_current_LA())

LA_soll_vec2 = np.full_like(t_vec,turbine_in.get_current_LA())
LA_soll_vec2[500:] = 0
# LA_soll_vec2[500:1000]   = 0.
# LA_soll_vec2[1000:1500]  = 1. 
# LA_soll_vec2[1500:2000]  = 0.
# LA_soll_vec2[2000:2500]  = 0.5 


In [9]:
%matplotlib qt5
# Con_T_ime loop

# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(2,1)
fig1.suptitle(str(0) +' s / '+str(round(t_vec[-1],2)) + ' s' )
axs1[0].set_title('Pressure distribution in pipeline')
axs1[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[0].set_ylabel(r'$p$ ['+pUnit_conv+']')
axs1[1].set_title('Flux distribution in pipeline')
axs1[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[1].set_ylabel(r'$Q$ [$\mathrm{m}^3 / \mathrm{s}$]')
lo_p,       = axs1[0].plot(Pip_x_vec,pressure_conversion(p_old,pUnit_calc, pUnit_conv),marker='.')
lo_q,       = axs1[1].plot(Pip_x_vec,Q_old,marker='.')
lo_pmin,    = axs1[0].plot(Pip_x_vec,pipe.get_lowest_pressure_per_node(disp_flag=True),c='red')
lo_pmax,    = axs1[0].plot(Pip_x_vec,pipe.get_highest_pressure_per_node(disp_flag=True),c='red')
lo_qmin,    = axs1[1].plot(Pip_x_vec,pipe.get_lowest_flux_per_node(),c='red')
lo_qmax,    = axs1[1].plot(Pip_x_vec,pipe.get_highest_flux_per_node(),c='red')

axs1[0].autoscale()
axs1[1].autoscale()

fig1.tight_layout()
fig1.show()
plt.pause(1)


In [10]:
convergence_parameters = [p_old[-2],v_old[-2],Pip_dia,Pip_area,Pip_angle,Pip_f_D,Pip_pw_vel,rho,Pip_dt]

# loop through Con_T_ime steps of the pipeline
for it_pipe in range(1,nt+1):

    turbine_in.update_LA(LA_soll_vec2[it_pipe])
    turbine_in.set_pressure(Tur_p_nenn)
    Q_in_vec[it_pipe] = turbine_in.get_current_Q()
    reservoir.set_influx(Q_in_vec[it_pipe])

# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    # set initial condition for the reservoir Con_T_ime evolution calculted with e-RK4
    reservoir.set_pressure(p_old[0],display_warning=False)
    reservoir.set_outflux(Q_old[0],display_warning=False)
    # calculate the Con_T_ime evolution of the reservoir level within each pipeline timestep to avoid runaway numerical error
    for it_res in range(Res_nt):
        reservoir.timestep_reservoir_evolution()                                                             
    level_vec[it_pipe] = reservoir.get_current_level()                                                 
    volume_vec[it_pipe] = reservoir.get_current_volume()                                                 

    # get the control variable
    level_control.update_control_variable(level_vec[it_pipe])
    LA_soll_vec[it_pipe] = level_control.get_current_control_variable()
    
    # change the Leitapparatöffnung  based on the target value
    turbine.update_LA(LA_soll_vec[it_pipe])
    LA_ist_vec[it_pipe] = turbine.get_current_LA()

    # set boundary condition for the next timestep of the characterisCon_T_ic method
    turbine.set_pressure(p_old[-1])
    convergence_parameters[0] = p_old[-2]
    convergence_parameters[1] = v_old[-2]
    turbine.converge(convergence_parameters)
    p_boundary_res[it_pipe] = reservoir.get_current_pressure()
    v_boundary_tur[it_pipe] = 1/Pip_area*turbine.get_current_Q()
    Q_boundary_tur[it_pipe] = turbine.get_current_Q()

    # the the boundary condition in the pipe.object and thereby calculate boundary pressure at turbine
    pipe.set_boundary_conditions_next_timestep(p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    pipe.v[0] = (0.8*pipe.v[0]+0.2*reservoir.get_current_outflux()/Res_area_out)
    p_boundary_tur[it_pipe] = pipe.get_current_pressure_distribution()[-1]
    v_boundary_res[it_pipe] = pipe.get_current_velocity_distribution()[0]
    Q_boundary_res[it_pipe] = pipe.get_current_flux_distribution()[0]

    # perform the next timestep via the characterisCon_T_ic method
    pipe.timestep_characteristic_method()

    # prepare for next loop
    p_old = pipe.get_current_pressure_distribution()
    v_old = pipe.get_current_velocity_distribution()
    Q_old = pipe.get_current_flux_distribution()

    # plot some stuff
        # remove line-objects to autoscale axes (there is definetly a better way, but this works ¯\_(ツ)_/¯ )
    lo_p.remove()
    lo_pmin.remove()
    lo_pmax.remove()
    lo_q.remove()
    lo_qmin.remove()
    lo_qmax.remove()
        # plot new pressure and velocity distribution in the pipeline
    lo_p, = axs1[0].plot(Pip_x_vec,pipe.get_current_pressure_distribution(disp_flag=True),marker='.',c='blue')
    lo_pmin, = axs1[0].plot(Pip_x_vec,pipe.get_lowest_pressure_per_node(disp_flag=True),c='red')
    lo_pmax, = axs1[0].plot(Pip_x_vec,pipe.get_highest_pressure_per_node(disp_flag=True),c='red')
    lo_q, = axs1[1].plot(Pip_x_vec,pipe.get_current_flux_distribution(),marker='.',c='blue')
    lo_qmin, = axs1[1].plot(Pip_x_vec,pipe.get_lowest_flux_per_node(),c='red')
    lo_qmax, = axs1[1].plot(Pip_x_vec,pipe.get_highest_flux_per_node(),c='red')
    fig1.suptitle(str(round(t_vec[it_pipe],2))+ ' s / '+str(round(t_vec[-1],2)) + ' s' )
    fig1.canvas.draw()
    fig1.tight_layout()
    fig1.show()
    plt.pause(0.001)    

In [11]:
# plot Con_T_ime evolution of boundary pressure and velocity as well as the reservoir level

fig2,axs2 = plt.subplots(1,1)
axs2.set_title('Level and Volume reservoir')
axs2.plot(t_vec,level_vec,label='level')
axs2.set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2.set_ylabel(r'$h$ [m]')
x_twin_00 = axs2.twinx()
x_twin_00.set_ylabel(r'$V$ [$\mathrm{m}^3$]')
x_twin_00.plot(t_vec,volume_vec)
axs2.legend()

fig2,axs2 = plt.subplots(1,1)
axs2.set_title('LA')
axs2.plot(t_vec,100*LA_soll_vec,label='Target')
axs2.plot(t_vec,100*LA_ist_vec,label='Actual')
axs2.set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2.set_ylabel(r'$LA$ [%]')
axs2.legend()

fig2,axs2 = plt.subplots(1,1)
axs2.set_title('Pressure reservoir and turbine')
axs2.plot(t_vec,pressure_conversion(p_boundary_res,pUnit_calc, pUnit_conv),label='Reservoir')
axs2.plot(t_vec,pressure_conversion(p_boundary_tur,pUnit_calc, pUnit_conv),label='Turbine')
axs2.set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2.set_ylabel(r'$p$ ['+pUnit_conv+']')
axs2.legend()

fig2,axs2 = plt.subplots(1,1)
axs2.set_title('Fluxes')
axs2.plot(t_vec,Q_boundary_res,label='Outflux')
axs2.plot(t_vec,Q_in_vec,label='Influx')
axs2.plot(t_vec,Q_boundary_tur,label='Flux Turbine')
axs2.set_ylim(-2*Tur_Q_nenn,+2*Tur_Q_nenn)
axs2.set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2.set_ylabel(r'$Q$ [$\mathrm{m}^3/\mathrm{s}$]')
axs2.legend()

fig2,axs2 = plt.subplots(1,1)
axs2.set_title('Min and Max Pressure')
axs2.plot(Pip_x_vec,pipe.get_lowest_pressure_per_node(disp_flag=True),c='red')
axs2.plot(Pip_x_vec,pipe.get_highest_pressure_per_node(disp_flag=True),c='red')
axs2.set_xlabel(r'$x$ [$\mathrm{m}$]')
axs2.set_ylabel(r'$p$ ['+pUnit_conv+']')

fig2,axs2 = plt.subplots(1,1)
axs2.set_title('Min and Max Fluxes')
axs2.plot(Pip_x_vec,pipe.get_lowest_flux_per_node(),c='red')
axs2.plot(Pip_x_vec,pipe.get_highest_flux_per_node(),c='red')
axs2.set_xlabel(r'$x$ [$\mathrm{m}$]')
axs2.set_ylabel(r'$Q$ [$\mathrm{m}^3/\mathrm{s}$]')

# axs2[0,1].legend()
# axs2[1,0].legend()
# axs2[1,1].legend()
# # axs2[2,0].legend()
# # axs2[2,1].legend()


fig2.tight_layout()
plt.show()